TD3 Model
@inproceedings{fujimoto2018addressing,
  title={Addressing Function Approximation Error in Actor-Critic Methods},
  author={Fujimoto, Scott and Hoof, Herke and Meger, David},
  booktitle={International Conference on Machine Learning},
  pages={1582--1591},
  year={2018}
}

In [5]:
import copy
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F


device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Implementation of Twin Delayed Deep Deterministic Policy Gradients (TD3)
# Paper: https://arxiv.org/abs/1802.09477


class Actor(nn.Module):
	def __init__(self, state_dim, action_dim, max_action):
		super(Actor, self).__init__()

		self.l1 = nn.Linear(state_dim, 256)
		self.l2 = nn.Linear(256, 256)
		self.l3 = nn.Linear(256, action_dim)
		
		self.max_action = max_action
		

	def forward(self, state):
		a = F.relu(self.l1(state))
		a = F.relu(self.l2(a))
		return self.max_action * torch.tanh(self.l3(a))


class Critic(nn.Module):
	def __init__(self, state_dim, action_dim):
		super(Critic, self).__init__()

		# Q1 architecture
		self.l1 = nn.Linear(state_dim + action_dim, 256)
		self.l2 = nn.Linear(256, 256)
		self.l3 = nn.Linear(256, 1)

		# Q2 architecture
		self.l4 = nn.Linear(state_dim + action_dim, 256)
		self.l5 = nn.Linear(256, 256)
		self.l6 = nn.Linear(256, 1)


	def forward(self, state, action):
		sa = torch.cat([state, action], 1)

		q1 = F.relu(self.l1(sa))
		q1 = F.relu(self.l2(q1))
		q1 = self.l3(q1)

		q2 = F.relu(self.l4(sa))
		q2 = F.relu(self.l5(q2))
		q2 = self.l6(q2)
		return q1, q2


	def Q1(self, state, action):
		sa = torch.cat([state, action], 1)

		q1 = F.relu(self.l1(sa))
		q1 = F.relu(self.l2(q1))
		q1 = self.l3(q1)
		return q1


class TD3(object):
	def __init__(
		self,
		state_dim,
		action_dim,
		max_action,
		discount=0.99,
		tau=0.005,
		policy_noise=0.2,
		noise_clip=0.5,
		policy_freq=2
	):

		self.actor = Actor(state_dim, action_dim, max_action).to(device)
		self.actor_target = copy.deepcopy(self.actor)
		self.actor_optimizer = torch.optim.Adam(self.actor.parameters(), lr=3e-4)

		self.critic = Critic(state_dim, action_dim).to(device)
		self.critic_target = copy.deepcopy(self.critic)
		self.critic_optimizer = torch.optim.Adam(self.critic.parameters(), lr=3e-4)

		self.max_action = max_action
		self.discount = discount
		self.tau = tau
		self.policy_noise = policy_noise
		self.noise_clip = noise_clip
		self.policy_freq = policy_freq

		self.total_it = 0


	def select_action(self, state):
		state = torch.FloatTensor(state.reshape(1, -1)).to(device)
		return self.actor(state).cpu().data.numpy().flatten()


	def train(self, replay_buffer, batch_size=100):
		self.total_it += 1

		# Sample replay buffer 
		state, action, next_state, reward, not_done = replay_buffer.sample(batch_size)

		with torch.no_grad():
			# Select action according to policy and add clipped noise
			noise = (
				torch.randn_like(action) * self.policy_noise
			).clamp(-self.noise_clip, self.noise_clip)
			
			next_action = (
				self.actor_target(next_state) + noise
			).clamp(-self.max_action, self.max_action)

			# Compute the target Q value
			target_Q1, target_Q2 = self.critic_target(next_state, next_action)
			target_Q = torch.min(target_Q1, target_Q2)
			target_Q = reward + not_done * self.discount * target_Q

		# Get current Q estimates
		current_Q1, current_Q2 = self.critic(state, action)

		# Compute critic loss
		critic_loss = F.mse_loss(current_Q1, target_Q) + F.mse_loss(current_Q2, target_Q)

		# Optimize the critic
		self.critic_optimizer.zero_grad()
		critic_loss.backward()
		self.critic_optimizer.step()

		# Delayed policy updates
		if self.total_it % self.policy_freq == 0:

			# Compute actor losse
			actor_loss = -self.critic.Q1(state, self.actor(state)).mean()
			
			# Optimize the actor 
			self.actor_optimizer.zero_grad()
			actor_loss.backward()
			self.actor_optimizer.step()

			# Update the frozen target models
			for param, target_param in zip(self.critic.parameters(), self.critic_target.parameters()):
				target_param.data.copy_(self.tau * param.data + (1 - self.tau) * target_param.data)

			for param, target_param in zip(self.actor.parameters(), self.actor_target.parameters()):
				target_param.data.copy_(self.tau * param.data + (1 - self.tau) * target_param.data)


	def save(self, filename):
		torch.save(self.critic.state_dict(), filename + "_critic")
		torch.save(self.critic_optimizer.state_dict(), filename + "_critic_optimizer")
		
		torch.save(self.actor.state_dict(), filename + "_actor")
		torch.save(self.actor_optimizer.state_dict(), filename + "_actor_optimizer")


	def load(self, filename):
		self.critic.load_state_dict(torch.load(filename + "_critic"))
		self.critic_optimizer.load_state_dict(torch.load(filename + "_critic_optimizer"))
		self.critic_target = copy.deepcopy(self.critic)

		self.actor.load_state_dict(torch.load(filename + "_actor"))
		self.actor_optimizer.load_state_dict(torch.load(filename + "_actor_optimizer"))
		self.actor_target = copy.deepcopy(self.actor)

Replay Buffer (from the same repo as TD3 Model)
@inproceedings{fujimoto2018addressing,
  title={Addressing Function Approximation Error in Actor-Critic Methods},
  author={Fujimoto, Scott and Hoof, Herke and Meger, David},
  booktitle={International Conference on Machine Learning},
  pages={1582--1591},
  year={2018}
}

In [14]:
class ReplayBuffer(object):
	def __init__(self, state_dim, action_dim, max_size=int(1e6)):
		self.max_size = max_size
		self.ptr = 0
		self.size = 0

		self.state = np.zeros((max_size, state_dim))
		self.action = np.zeros((max_size, action_dim))
		self.next_state = np.zeros((max_size, state_dim))
		self.reward = np.zeros((max_size, 1))
		self.not_done = np.zeros((max_size, 1))

		self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


	def add(self, state, action, next_state, reward, done):
		self.state[self.ptr] = state
		self.action[self.ptr] = action
		self.next_state[self.ptr] = next_state
		self.reward[self.ptr] = reward
		self.not_done[self.ptr] = 1. - done

		self.ptr = (self.ptr + 1) % self.max_size
		self.size = min(self.size + 1, self.max_size)


	def sample(self, batch_size):
		ind = np.random.randint(0, self.size, size=batch_size)

		return (
			torch.FloatTensor(self.state[ind]).to(self.device),
			torch.FloatTensor(self.action[ind]).to(self.device),
			torch.FloatTensor(self.next_state[ind]).to(self.device),
			torch.FloatTensor(self.reward[ind]).to(self.device),
			torch.FloatTensor(self.not_done[ind]).to(self.device)
		)

In [22]:
#Code for policy evaluation
# Runs policy for X episodes and returns average reward
# A fixed seed is used for the eval environment
def eval_policy(policy, env_name, seed, eval_episodes=10):
	eval_env = gym.make(env_name)
	eval_env.seed(seed + 100) #get a different seed from before

	avg_reward = 0.
	for _ in range(eval_episodes):
		state, done = eval_env.reset(), False
		while not done:
			action = policy.select_action(np.array(state))
			state, reward, done, _ = eval_env.step(action)
			avg_reward += reward

	avg_reward /= eval_episodes

	print("---------------------------------------")
	print(f"Evaluation over {eval_episodes} episodes: {avg_reward:.3f}")
	print("---------------------------------------")
	return avg_reward

In [6]:
!pip3 install numpngw
from numpngw import write_apng
import gym

In [17]:
#Create the environment
env_id = "Pendulum-v0"
env = gym.make(env_id)
state_dim = env.observation_space.shape[-1]
action_dim = env.action_space.shape[-1]
max_action=env.max_torque
print('State dimension', state_dim)
print('Action dimension', action_dim)
print('Max action', max_action)
print('Max number of episodes', env._max_episode_steps)


State dimension 3
Action dimension 1
Max action 2.0
Max number of episodes 200


In [19]:
#Seed environment, torch, and numpy for consistent results 
#Will need to find an optimal seed eventually
seed=0 # Sets Gym, PyTorch and Numpy seeds
env.seed(seed)
torch.manual_seed(seed)
np.random.seed(seed)

In [20]:
#Define all the hyperparameters (currently default from the repo this code was taken)
start_timesteps = 25e3 # Time steps initial random policy is used
expl_noise = 0.1  # Std of Gaussian exploration noise
policy_noise = 0.2 # Noise added to target policy during critic update
batch_size = 256 # Batch size for both actor and critic
noise_clip = 0.5 # Range to clip target policy noise
policy_freq = 2 # Frequency of delayed policy updates
tau = 0.005 # Target network update rate
discount = 0.99 # Discount factor
eval_freq = 5e3 # How often (time steps) we evaluate

max_timesteps = 1e6 # Max time steps to run environment


In [21]:
	kwargs = {
		"state_dim": state_dim,
		"action_dim": action_dim,
		"max_action": max_action,
		"discount": discount,
		"tau": tau,
	}

#Init model with the appropriate env variables and previously defined hyperparameters

# Target policy smoothing is scaled wrt the action scale
kwargs["policy_noise"] = policy_noise * max_action
kwargs["noise_clip"] = noise_clip * max_action
kwargs["policy_freq"] = policy_freq
policy = TD3(**kwargs)

In [23]:
#To save models while training:
save_model = True
file_name = 'Test'

In [ ]:
#Training
replay_buffer = ReplayBuffer(state_dim, action_dim)
	
# Evaluate untrained policy
evaluations = []

state, done = env.reset(), False
episode_reward = 0
episode_timesteps = 0
episode_num = 0

for t in range(int(max_timesteps)):
  
  episode_timesteps += 1

  # Select action randomly or according to policy
  if t < start_timesteps:
    action = env.action_space.sample()
  else:
    action = (
      policy.select_action(np.array(state))
      + np.random.normal(0, max_action * expl_noise, size=action_dim)
    ).clip(-max_action, max_action)

  # Perform action
  next_state, reward, done, _ = env.step(action) 
  done_bool = float(done) if episode_timesteps < env._max_episode_steps else 0

  # Store data in replay buffer
  replay_buffer.add(state, action, next_state, reward, done_bool)

  state = next_state
  episode_reward += reward

  # Train agent after collecting sufficient data
  if t >= start_timesteps:
    policy.train(replay_buffer, batch_size)

  if done: 
    # +1 to account for 0 indexing. +0 on ep_timesteps since it will increment +1 even if done=True
    print(f"Total T: {t+1} Episode Num: {episode_num+1} Episode T: {episode_timesteps} Reward: {episode_reward:.3f}")
    # Reset environment
    state, done = env.reset(), False
    episode_reward = 0
    episode_timesteps = 0
    episode_num += 1 

  # Evaluate episode
  if (t + 1) % eval_freq == 0:
    evaluations.append(eval_policy(policy, env_id, seed))
    np.save(f"results_{t+1}_{file_name}", evaluations)
    if save_model:
       policy.save(f"models_{t+1}_{file_name}")
      #  files.download(f'{name}.zip') 

Total T: 200 Episode Num: 1 Episode T: 200 Reward: -1578.401
Total T: 400 Episode Num: 2 Episode T: 200 Reward: -971.775
Total T: 600 Episode Num: 3 Episode T: 200 Reward: -1599.040
Total T: 800 Episode Num: 4 Episode T: 200 Reward: -1009.411
Total T: 1000 Episode Num: 5 Episode T: 200 Reward: -1711.823
Total T: 1200 Episode Num: 6 Episode T: 200 Reward: -1493.793
Total T: 1400 Episode Num: 7 Episode T: 200 Reward: -862.868
Total T: 1600 Episode Num: 8 Episode T: 200 Reward: -1710.885
Total T: 1800 Episode Num: 9 Episode T: 200 Reward: -911.380
Total T: 2000 Episode Num: 10 Episode T: 200 Reward: -1552.590
Total T: 2200 Episode Num: 11 Episode T: 200 Reward: -1280.946
Total T: 2400 Episode Num: 12 Episode T: 200 Reward: -918.346
Total T: 2600 Episode Num: 13 Episode T: 200 Reward: -1606.409
Total T: 2800 Episode Num: 14 Episode T: 200 Reward: -1249.019
Total T: 3000 Episode Num: 15 Episode T: 200 Reward: -1046.533
Total T: 3200 Episode Num: 16 Episode T: 200 Reward: -1159.737
Total T: 